In [1]:
pip install scx>=1.3.0 #Note: This may throw an Error in Google Colab since Colab uses an outdated duckdb package. This file should still work in Colab without issues.

Note: you may need to restart the kernel to use updated packages.


<h1>Eco Pants P3</h1>
<p class="MsoNormal" style="margin-bottom: 17.0pt; line-height: normal;">You presented the model’s results to Barbara, who pointed out that you didn't include the set-up cost to be paid to a supplier every time you place a new order. Barbara requests you adjust the model to include these costs.</p>
<p><b>Table 4:</b><strong> Set-up cost per supplier (in $ per order).</strong></p>
<table width="444" height="66" style="height: 83px; width: 378px;">
<tbody>
<tr style="height: 51.2px;">
<td style="text-align: center; border: 1px solid; width: 166.891px; height: 25.6px;"></td>
<td style="text-align: center; border: 1px solid; width: 197.109px; height: 25.6px;">Set-up cost</td>
</tr>
<tr>
<td style="text-align: center; border: 1px solid; width: 166.891px;">Supplier 1</td>
<td style="text-align: center; border: 1px solid; width: 197.109px;">1500</td>
</tr>
<tr>
<td style="text-align: center; border: 1px solid; width: 166.891px;">Supplier 2</td>
<td style="text-align: center; border: 1px solid; width: 197.109px;">2500</td>
</tr>
<tr>
<td style="text-align: center; border: 1px solid; width: 166.891px;">Supplier 3</td>
<td style="text-align: center; border: 1px solid; width: 197.109px;">800</td>
</tr>
<tr>
<td style="text-align: center; border: 1px solid; width: 166.891px;">Supplier 4</td>
<td style="text-align: center; border: 1px solid; width: 197.109px;">1200</td>
</tr>
</tbody>
</table>
<p><strong style="background: transparent; margin-top: 0pt; margin-bottom: 0pt;"><span></span></strong></p>
<p><strong style="background: transparent; margin-top: 0pt; margin-bottom: 0pt;"><span>Adjust your model from Part 2 to incorporate suppliers' set-up costs.</span></strong></p>
<a href="https://youtu.be/ArhPAOxwdG4">Eco Pants P3 Video Walkthrough Link</a><br/>
<iframe width="560" height="315"
    src="https://www.youtube.com/embed/ArhPAOxwdG4" frameborder="0" allowfullscreen>
</iframe>

In [2]:
from scx.optimize import Model

In [3]:

supply = [
    {
        'name': 'S1',
        'capacity': 500,
        'location': 'local',
        'setup_cost': 1500,
    },
    {
        'name': 'S2',
        'capacity': 300,
        'location': 'local',
        'setup_cost': 2500,
    },
    {
        'name': 'S3',
        'capacity': 0,
        'location': 'foreign',
        'setup_cost': 800,
    },
    {
        'name': 'S4',
        'capacity': 600,
        'location': 'local',
        'setup_cost': 1200,
    },
]

demand = [
    {
        'name':'F1',
        'demand':396
    },
    {
        'name':'F2',
        'demand':255
    },
    {
        'name':'F3',
        'demand':289
    },
    {
        'name':'F4',
        'demand':15
    },
    {
        'name':'F5',
        'demand':61
    },
]

cost = [
    {
        'material_name': 'F1',
        'supplier_name': 'S1',
        'cost': 167
    },
    {
        'material_name': 'F1',
        'supplier_name': 'S2',
        'cost': 154
    },
    {
        'material_name': 'F1',
        'supplier_name': 'S3',
        'cost': 70
    },
    {
        'material_name': 'F1',
        'supplier_name': 'S4',
        'cost': 68
    },
    {
        'material_name': 'F2',
        'supplier_name': 'S1',
        'cost': 63
    },
    {
        'material_name': 'F2',
        'supplier_name': 'S2',
        'cost': 196
    },
    {
        'material_name': 'F2',
        'supplier_name': 'S3',
        'cost': 170
    },
    {
        'material_name': 'F2',
        'supplier_name': 'S4',
        'cost': 87
    },
    {
        'material_name': 'F3',
        'supplier_name': 'S1',
        'cost': 93
    },
    {
        'material_name': 'F3',
        'supplier_name': 'S2',
        'cost': 151
    },
    {
        'material_name': 'F3',
        'supplier_name': 'S3',
        'cost': 165
    },
    {
        'material_name': 'F3',
        'supplier_name': 'S4',
        'cost': 73
    },
    {
        'material_name': 'F4',
        'supplier_name': 'S1',
        'cost': 148
    },
    {
        'material_name': 'F4',
        'supplier_name': 'S2',
        'cost': 136
    },
    {
        'material_name': 'F4',
        'supplier_name': 'S3',
        'cost': 72
    },
    {
        'material_name': 'F4',
        'supplier_name': 'S4',
        'cost': 64
    },
    {
        'material_name': 'F5',
        'supplier_name': 'S1',
        'cost': 189
    },
    {
        'material_name': 'F5',
        'supplier_name': 'S2',
        'cost': 106
    },
    {
        'material_name': 'F5',
        'supplier_name': 'S3',
        'cost': 195
    },
    {
        'material_name': 'F5',
        'supplier_name': 'S4',
        'cost': 125
    },
]

In [4]:
for c in cost:
    c['amt']=Model.variable(name=f"{c['supplier_name']}__{c['material_name']}__amt", lowBound=0)

In [5]:
for s in supply:
    s['use']=Model.variable(name=f"{s['name']}__use", cat="Binary")

In [6]:
M=99999999

In [7]:
# Initialize the model
my_model = Model(name="EcoPantsP1", sense='minimize')


# Add the Objective Fn
my_model.add_objective(
    fn=Model.sum([c['amt']*c['cost'] for c in cost]) + Model.sum([s['setup_cost']*s['use'] for s in supply]),
)

# Add Constraints
## Demand Constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}__demand",
        fn=Model.sum([c['amt'] for c in cost if c['material_name']==d['name']]) >= d['demand'],
    )

## Supply Constraints
for s in supply:
    # Capacity Constraint
    my_model.add_constraint(
        name=f"{s['name']}__supply",
        fn=Model.sum([c['amt'] for c in cost if c['supplier_name']==s['name']]) <= (s['capacity'] if s['location']=='local' else 0),
    )
    # Setup Cost Constraint
    my_model.add_constraint(
        name=f"{s['name']}__setup",
        fn=Model.sum([c['amt'] for c in cost if c['supplier_name']==s['name']]) <= s['use']*M,
    )

# Solve the model
my_model.solve()

In [8]:
my_model.show_formulation()

EcoPantsP1:
MINIMIZE
167*S1__F1__amt + 63*S1__F2__amt + 93*S1__F3__amt + 148*S1__F4__amt + 189*S1__F5__amt + 1500*S1__use + 154*S2__F1__amt + 196*S2__F2__amt + 151*S2__F3__amt + 136*S2__F4__amt + 106*S2__F5__amt + 2500*S2__use + 70*S3__F1__amt + 170*S3__F2__amt + 165*S3__F3__amt + 72*S3__F4__amt + 195*S3__F5__amt + 800*S3__use + 68*S4__F1__amt + 87*S4__F2__amt + 73*S4__F3__amt + 64*S4__F4__amt + 125*S4__F5__amt + 1200*S4__use + 0.0
SUBJECT TO
F1__demand: S1__F1__amt + S2__F1__amt + S3__F1__amt + S4__F1__amt >= 396

F2__demand: S1__F2__amt + S2__F2__amt + S3__F2__amt + S4__F2__amt >= 255

F3__demand: S1__F3__amt + S2__F3__amt + S3__F3__amt + S4__F3__amt >= 289

F4__demand: S1__F4__amt + S2__F4__amt + S3__F4__amt + S4__F4__amt >= 15

F5__demand: S1__F5__amt + S2__F5__amt + S3__F5__amt + S4__F5__amt >= 61

S1__supply: S1__F1__amt + S1__F2__amt + S1__F3__amt + S1__F4__amt
 + S1__F5__amt <= 500

S1__setup: S1__F1__amt + S1__F2__amt + S1__F3__amt + S1__F4__amt + S1__F5__amt
 - 99999999 S1__u

In [9]:
# Show the outputs
my_model.show_outputs()

{'objective': 78595.0,
 'status': 'Optimal',
 'variables': {'S1__F1__amt': 0.0,
               'S1__F2__amt': 255.0,
               'S1__F3__amt': 161.0,
               'S1__F4__amt': 0.0,
               'S1__F5__amt': 0.0,
               'S1__use': 1.0,
               'S2__F1__amt': 0.0,
               'S2__F2__amt': 0.0,
               'S2__F3__amt': 0.0,
               'S2__F4__amt': 0.0,
               'S2__F5__amt': 0.0,
               'S2__use': 0.0,
               'S3__F1__amt': 0.0,
               'S3__F2__amt': 0.0,
               'S3__F3__amt': 0.0,
               'S3__F4__amt': 0.0,
               'S3__F5__amt': 0.0,
               'S3__use': 0.0,
               'S4__F1__amt': 396.0,
               'S4__F2__amt': 0.0,
               'S4__F3__amt': 128.0,
               'S4__F4__amt': 15.0,
               'S4__F5__amt': 61.0,
               'S4__use': 1.0}}
